In [ ]:
import plotly.io as pio
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
init_notebook_mode(connected=True)  # initiate notebook for offline plot

# Tutorial - angles and inter-nodes sequences alignement

In [ ]:
import numpy as np
from dtw import DTW
from dtw.tasks.search_free_ends import brute_force_free_ends_search
from dtw.metrics import mixed_dist
from dtw.visu import sequences_table
from dtw.visu import plot_sequences
from dtw.visu import plot_aligned_sequences

## Reference and test sequences

These are _angles_ and _inter-nodes_ sequences measured manually and automatically (computer vision).

In [ ]:
seq_test = np.array([[96, 163, 137, 113, 24, 170, 152, 137, 255, 148, 111, 16, 334, 160, 94, 116, 144, 132, 145], [50, 60, 48, 19, 31, 0, 37, 20, 31, 25, 7, 1, 51, 29, 26, 16, 22, 12, 23]]).T
seq_ref = np.array([[96, 163, 137, 137, 170, 152, 137, 132, 123, 148, 127, 191, 143, 160, 94, 116, 144, 132, 145], [50, 60, 48, 50, 0, 37, 20, 0, 31, 25, 8, 27, 24, 29, 26, 16, 22, 12, 23 ]]).T

## Initialize the DTW object

To performs this alignment, we use the "merge-split" constraint to account for organ under or over segmentation.
We also use a "mixed distance" that we need to initialize.

### Mixed-distance initialization

In [ ]:
max_ref = np.max(seq_ref[:, 1])
max_test = np.max(seq_test[:, 1])
# Variables associated to 'mixed-distance':
mtype = [True, False]
mspread = [1, max(max_ref, max_test)]
mweight = [0.5, 0.5]

### Dynamic Time Warpping algorithm

In [ ]:
dtw = DTW(seq_test, seq_ref, names=['angles', 'inter-nodes'], constraints='merge_split', ldist=mixed_dist,
          mixed_type=mtype, mixed_spread=mspread, mixed_weight=mweight)

Using representation methods provided by the `dtw` package we can represent the two sequences in tables using the `sequences_table` method:

In [ ]:
fig = sequences_table(dtw)
iplot(fig)

We now represent the two sequences of _angles_ and _inter-nodes_ as scatter plots using the `plot_sequences` method:

In [ ]:
fig = plot_sequences(dtw)
iplot(fig)

## DTW alignment

### Free-ends brute force search

In [ ]:
free_ends, norm_dist = brute_force_free_ends_search(dtw)
print(f"Found free-ends {free_ends} with a normalized distance of {norm_dist}.")

### Alignment with selected free-ends

Set the free-ends values to use in `run` method:

In [ ]:
dtw.free_ends = free_ends

Let's start by running the DTW algorithm and find the best alignment with initialization.

In [ ]:
ndist, path, length, ndistarray, backpointers = dtw.run()

### Summaries

To get the results of the alginment:

In [ ]:
rd = dtw.get_results()

In [ ]:
rd

To get the type of events predicted and their cost:

In [ ]:
print(f"Found event types:{rd['type']}")
print(f"With associated costs:{rd['cost']}")

In [ ]:
dtw.summarize()

Now we can represent the aligned sequences using the `plot_aligned_sequences` method:

In [ ]:
fig = plot_aligned_sequences(dtw)
iplot(fig)